#### Teachable Machine 추출 모델 테스트

In [1]:
!python --version

Python 3.9.21


In [2]:
pwd

'/home/oote/elthe/cell_detection'

In [1]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import glob
from matplotlib import pyplot as plt
import h5py
Image.ANTIALIAS = Image.LANCZOS

2025-01-27 01:55:46.406867: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-27 01:55:47.174025: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
f = h5py.File("keras_model3.h5", mode="r+")
model_config_string = f.attrs.get("model_config")
if model_config_string.find('"groups": 1,') != -1:
    model_config_string = model_config_string.replace('"groups": 1,', '')
    f.attrs.modify('model_config', model_config_string)
    f.flush()
    model_config_string = f.attrs.get("model_config")
    assert model_config_string.find('"groups": 1,') == -1

f.close()

In [4]:
# Load the model
model = load_model('keras_model3.h5',compile=False)

In [22]:
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
detection_result=[]
#detection_result='N'

image = Image.open('test/(150).PNG').convert('RGB')
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

image_array = np.asarray(image)
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
data[0] = normalized_image_array

prediction = model(data)
print(prediction)

if(prediction[0,0]>prediction[0,1]):
  #detection_result='Red'
  detection_result.append('Red')
  print('Result : ',detection_result)
else:
  #detection_result='Green'
  detection_result.append('Green')
  print('Result : ',detection_result)

tf.Tensor([[0.00031005 0.99968994]], shape=(1, 2), dtype=float32)
Result :  ['Red']


#### Red/Green 개수 출력

In [14]:
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
size = (224, 224)
detection_result = []

image_files = glob.glob('test/*.PNG')
red_images = []
for i in image_files:
  image = Image.open(i).convert('RGB')
  image = ImageOps.fit(image, size, Image.ANTIALIAS)

  image_array = np.asarray(image)
  normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
  data[0] = normalized_image_array

  prediction = model(data)

  if(prediction[0,0] < prediction[0,1]):
    detection_result.append('Red')
    red_images.append(i)
  else:
    detection_result.append('Green')

#result=detection_result.count('Green')+detection_result.count('Red')

print('Green : ',detection_result.count('Green'))
print('Red : ',detection_result.count('Red'))
print('red : ',red_images)

Green :  44
Red :  27
red :  ['test/(107).PNG', 'test/(200).PNG', 'test/(98).PNG', 'test/(102).PNG', 'test/(113).PNG', 'test/(106).PNG', 'test/(110).PNG', 'test/(116).PNG', 'test/(217).PNG', 'test/(97).PNG', 'test/(111).PNG', 'test/(94).PNG', 'test/(114).PNG', 'test/(100).PNG', 'test/(219).PNG', 'test/(105).PNG', 'test/(210).PNG', 'test/(115).PNG', 'test/(103).PNG', 'test/(112).PNG', 'test/(216).PNG', 'test/(197).PNG', 'test/(95).PNG', 'test/(104).PNG', 'test/(226).PNG', 'test/(101).PNG', 'test/(96).PNG']


In [8]:
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
size = (224, 224)
detection_result = []
labels=['Red','Green','Exception']

image_files = glob.glob('test/*.PNG')

for i in image_files:
  image = Image.open(i)
  image = ImageOps.fit(image, size, Image.ANTIALIAS)

  image_array = np.asarray(image)
  normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
  data[0] = normalized_image_array

  prediction = model.predict(data)
  idx=np.argmax(prediction)

  if(labels[idx]=='Green'):
    detection_result.append('Green')
  elif(labels[idx]=='Red'):
    detection_result.append('Red')
  elif(labels[idx]=='Exception'):
    detection_result.append('Exception')

  print("[ Result : ",labels[idx],"]")
  plt.imshow(image)
  plt.show()
  print("\n")

print("===============")
print('Green : ',detection_result.count('Green'))
print('Red : ',detection_result.count('Red'))
#print('Exception : ',detection_result.count('Exception'))
print("===============")

Green :  0
Red :  0


In [9]:
# classification by threshold
pred_y = [1 if e > threshold_fixed else 0 for e in error_df['Reconstruction_error'].values]

conf_matrix = metrics.confusion_matrix(error_df['True_class'], pred_y)
plt.figure(figsize=(7, 7))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Class'); plt.ylabel('True Class')
plt.show()

NameError: name 'error_df' is not defined

In [4]:
false_pos_rate, true_pos_rate, thresholds = metrics.roc_curve(error_df['True_class'], error_df['Reconstruction_error'])
roc_auc = metrics.auc(false_pos_rate, true_pos_rate,)

plt.plot(false_pos_rate, true_pos_rate, linewidth=5, label='AUC = %0.3f'% roc_auc)
plt.plot([0,1],[0,1], linewidth=5)

plt.xlim([-0.01, 1])
plt.ylim([0, 1.01])
plt.legend(loc='lower right')
plt.title('Receiver operating characteristic curve (ROC)')
plt.ylabel('True Positive Rate'); plt.xlabel('False Positive Rate')
plt.show()

NameError: name 'metrics' is not defined